<a href="https://colab.research.google.com/github/johnpolsh/inf721-tpfinal/blob/main/colab/Object_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
### Default imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

### Select back-end device

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

print(f"Using {device} as default device")

# Model
### Our archtecture definition

In [ ]:
import torch.nn

class Bottleneck():
    def __init__(self):
        pass

class OurObjectDetectionNet(nn.Module):
    def __init__(self):
        super.__init__(self, OurObjectDetectionNet)

        self.out == None
    
    def __forward_impl(self, x):
        return x

    def forward(self, x):
        x = self.__forward_impl(x)
        return x

our_model = OurObjectDetectionNet()

### MobileNet V2

In [ ]:
from torchvision.models import mobilenet_v2

mobilenet_model = mobilenet_v2(pretrained=True)

### Convert model for mobile

In [ ]:
import wget
import os.path

if not os.path.isfile("convert.py"):
    wget.download(
        "https://raw.githubusercontent.com/johnpolsh/inf721-tpfinal/main/colab/convert.py",
        "convert.py")

In [ ]:
from convert import convert_for_mobile

convert_for_mobile(mobilenet_model, "object_detection.plt")